<a href="https://colab.research.google.com/github/areias/bert_covid_sentiment/blob/main/bert_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2 MB 3.9 kB/s 
     |████████████████████████████████| 3.0 MB 31.8 MB/s 
     |████████████████████████████████| 2.9 MB 47.5 MB/s 
     |████████████████████████████████| 454 kB 82.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalli

In [6]:
!pip install  tensorflow_addons==0.11.2


     |████████████████████████████████| 1.1 MB 7.7 MB/s 


In [1]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [2]:
import tensorflow_addons
print(tensorflow_addons.__version__)

0.11.2


In [8]:
import sys
sys.path.append('drive/MyDrive/ct-bert')
sys.path.append('drive/MyDrive/ct-bert/tensorflow_models')


In [6]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 6.9 MB/s 


In [9]:
from official.utils.misc import distribution_utils
from official.nlp.bert import bert_models
from official.nlp.bert import configs as bert_configs
from official.nlp.bert import tokenization
from official.nlp.bert.input_pipeline import single_file_dataset

import os
import datetime
import time
import argparse
import logging
from tqdm import tqdm
import json
import tensorflow as tf
from utils.misc import ArgParseDefault, add_bool_arg, save_to_json
from config import PRETRAINED_MODELS
import collections
import pandas as pd

In [ ]:
VOCAB_PATH = 'vocabs'

In [ ]:
# start time
s_time = time.time()


In [10]:
from collections import namedtuple
arguments = namedtuple('arguments', ['run_name'])

args = arguments('run_2021-11-24_08-24-14_193220_test_run')
args

arguments(run_name='run_2021-11-24_08-24-14_193220_test_run')

In [29]:
# paths
run_dir = os.path.join('drive/MyDrive/ct-bert/data/finetune/',args.run_name,'crowdbreaks')
ts = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S_%f')
output_folder = os.path.join('drive/MyDrive/ct-bert/data/finetune/', args.run_name,'crowdbreaks', 'predictions', f'predictions_{ts}')
predictions_output_folder = os.path.join('drive/MyDrive/ct-bert/data/finetune/', args.run_name,'crowdbreaks','predictions', f'predictions_{ts}', 'predictions')

In [30]:
print(run_dir)
print(output_folder)
print(predictions_output_folder)

drive/MyDrive/ct-bert/data/finetune/run_2021-11-24_08-24-14_193220_test_run/crowdbreaks
drive/MyDrive/ct-bert/data/finetune/run_2021-11-24_08-24-14_193220_test_run/crowdbreaks/predictions/predictions_2021-11-24_13-57-04_869872
drive/MyDrive/ct-bert/data/finetune/run_2021-11-24_08-24-14_193220_test_run/crowdbreaks/predictions/predictions_2021-11-24_13-57-04_869872/predictions


In [32]:
if not os.path.isdir(predictions_output_folder):
    os.makedirs(predictions_output_folder)

In [34]:
def read_run_log(run_dir):
    with tf.io.gfile.GFile(os.path.join(run_dir, 'run_logs.json'), 'rb') as reader:
        run_log = json.loads(reader.read().decode('utf-8'))
    return run_log

In [35]:
# read configs
#logger.info(f'Reading run configs...')
run_log = read_run_log(run_dir)
pretrained_model_config_path = get_model_config_path(args)
model_config = get_model_config(pretrained_model_config_path)
max_seq_length = run_log['max_seq_length']
label_mapping = run_log['label_mapping']
num_labels = len(label_mapping)

NotFoundError: ignored